In [67]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import json
import os
from tps import ThinPlateSpline
from skspatial.objects import Plane, Points

In [68]:
import plotly.graph_objects as go
import plotly.express as px
import pyvista as pv
# pip install --upgrade trame-vtk
pv.set_jupyter_backend('trame')

In [69]:
from rotation_utils import *

In [70]:
os.getcwd()

'd:\\Docs_ASUS\\WORK\\hip\\MotionStudy\\MotionData\\001'

<div class="alert alert-block alert-info">
<b>REMEMBER:</b> OSIM works in meters, 3DSlicer works in milimeters - watch scaling of data!
</div>

## Import OSIM Markers from XML file

In [71]:
# parse the .xml with markers in body frames
markers_tree=ET.parse("markers_and_bone_markers_in_bodies.xml")
markers_root = markers_tree.getroot()
print(markers_root)

<Element 'OpenSimDocument' at 0x000001A13D1E6310>


In [72]:
# extract markers and create a list of dictionaries
mrkrs = []
for Marker in markers_root.iter('Marker'):
    dic = Marker.attrib
    for child in Marker:
        tagg = child.tag
        dic[tagg] = child.text
    mrkrs.append(dic)
len(mrkrs)

112

In [73]:
# split information in preparation for data frame
mrkrs_new = []
keys = ['name', 'socket_parent_frame', 'location']
for mrkr in mrkrs:
    dic = {}
    for key in keys:
        if key == 'name':
            dic[key] = mrkr[key]
        if key == 'socket_parent_frame':
            if '/bodyset' in mrkr[key]:
                mrkr[key] = mrkr[key][9:]
            elif '/' in mrkr[key]:
                mrkr[key] = mrkr[key][1:]
            dic[key] = mrkr[key]
        if key == 'location':
            a = mrkr[key].split()
            a = [float(i) for i in a]
            dic['r'] = a[0]*1000
            dic['a'] = a[1]*1000
            dic['s'] = a[2]*1000
    mrkrs_new.append(dic)

In [74]:
# create the dataframe that contains all information
df_osim_bone_markers = pd.DataFrame.from_dict(mrkrs_new)
df_osim_bone_markers.set_index('name', inplace=True)
df_osim_bone_markers.tail()
# export the dataframe to CVS
# df_markers.to_csv('model_update/markers_in_child.csv')

,socket_parent_frame,r,a,s
name,,,,
fibula_l_as,tibia_l,-5.585158,-62.582719,-19.747573
pelvis_origin,pelvis,0.000000,0.000000,0.000000
fibula_r_sup,tibia_r,-22.723540,-43.084800,29.005260
fibula_l_sup,tibia_l,-22.723540,-43.084800,-29.005260
torso_origin_in_pelvis,pelvis,-100.700000,81.500000,0.000000


##  Import OSIM muscles and muscle wrapping surfaces from the Rajagopal model file

In [75]:
# Import and parse the original Rajagopal model with muscles
tree_model = ET.parse('RajagopalModified_generic_copy.osim')
root_model = tree_model.getroot()

#### Import muscles

In [76]:
# Extract the muscle path points
lst = []
for PathPointSet in root_model.iter('PathPointSet'):
    lst.append(PathPointSet)

lm_groups = []
for N in lst:
    n = N[0]
    l = []
    for el in n:
        dic = el.attrib
        for child in el:
            tagg = child.tag
            dic[tagg] = child.text
        l.append(dic)
    lm_groups.append(l)

In [77]:
# Split information in preparation for a dataframe
m_data = []
for muscle in lm_groups:
    for landmark in muscle:
        entry = {}
        lm = landmark['name']
        entry['label'] = lm

        msk = lm[:-3]
        entry['muscle'] = msk
  
        a = landmark['location'].split()
        a = [float(i) for i in a]
        entry['r'] = a[0]*1000
        entry['a'] = a[1]*1000
        entry['s'] = a[2]*1000
        
        body = landmark['socket_parent_frame']
        entry['description'] = body[9:]        
        m_data.append(entry)

In [78]:
# create the dataframe with all muscles and their bodies
df_muscles = pd.DataFrame(m_data)
df_muscles.head()

,label,muscle,r,a,s,description
0,addbrev_r-P1,addbrev_r,-19.10,-94.00,15.40,pelvis
1,addbrev_r-P2,addbrev_r,-2.00,-118.00,24.90,femur_r
2,addlong_r-P1,addlong_r,-7.58,-88.90,18.88,pelvis
3,addlong_r-P2,addlong_r,11.26,-239.37,15.83,femur_r
4,addmagDist_r-P1,addmagDist_r,-74.04,-127.67,39.82,pelvis


#### Import muscle wrapping objects

<div class="alert alert-block alert-info"> <b>NOTE</b> The output dataframe does not split translation and rotation into axes. </div>

In [79]:
# Extract the muscle path points
wrp_lst = []
for Body in root_model.iter('Body'):
    body_name = Body.get('name')
    objs = Body.iter('WrapCylinder')
    # print(objs)
    for obj in objs:
        ob = {}
        obj_name = obj.get('name')
        rotation = (obj.find('xyz_body_rotation').text).split()
        rotation = np.array([float(i) for i in rotation])
        translation = (obj.find('translation').text).split()
        translation = np.array([float(i) for i in translation])
        radius = float(obj.find('radius').text)
        half_length = 0.5*float(obj.find('length').text)
        ob['body'], ob['name'] = (body_name, obj_name)
        ob['rotation'], ob['translation'] = (rotation, translation*1000) # rotation is in radians
        ob['radius'], ob['half_length'] = (radius*1000, half_length*1000)
        wrp_lst.append(ob)

wrp_df = pd.DataFrame(wrp_lst)
wrp_df.tail(10)

,body,name,rotation,translation,radius,half_length
30,femur_l,AMmid_at_femshaft_l,"[-1.61139, -0.13656, 0.0]","[23.012500000000003, -160.71099999999998, -20....",21.4,60.0
31,femur_l,AMdist_at_femshaft_l,"[-1.71188, -0.186636, 0.0]","[31.606500000000004, -260.73600000000005, -9.3...",21.8,100.0
32,femur_l,AMisch_at_condyles_l,"[-1.71115, 0.463363, 0.0]","[-22.6511, -376.831, 3.15437]",40.0,120.0
33,femur_l,PECT_at_femshaft_l,"[-1.81295, -0.276344, 0.0]","[6.08573, -84.50290000000001, -30.4405]",15.0,25.0
34,tibia_l,GasLat_at_shank_l,"[-2.96723, 0.279725, -1.47812]","[-7.4, -74.0, 3.3]",55.0,50.0
35,tibia_l,GasMed_at_shank_l,"[-2.96723, -0.0279725, -1.47812]","[-7.4, -74.0, 3.3]",55.0,50.0
36,tibia_l,GR_at_condyles_l,"[0.0, 0.4, 0.0]","[-3.0, -20.0, 0.0]",36.0,50.0
37,tibia_l,SM_at_condyles_l,"[0.0, 0.1, 0.0]","[-1.0, -20.0, 0.0]",35.2,50.0
38,tibia_l,ST_at_condyles_l,"[0.0, 0.2, 0.0]","[-2.0, -20.5, 0.0]",42.5,50.0
39,tibia_l,BF_at_gastroc_l,"[0.0, 0.0, 0.0]","[-58.0, -60.0, 0.0]",30.0,75.0


In [80]:
# calculate center points for the location of radius and axis of the wrapping surface
# add them to the table

wrp_df['radius_point']=['']*wrp_df.shape[0]
wrp_df['axis_point']=['']*wrp_df.shape[0]

for i, row in wrp_df.iterrows():
    axes = np.array([[1,0,0],[0,1,0],[0,0,1]])
    angles = row['rotation']
    center = row['translation']

    x_rotation=rotation_matrix(axes[0], angles[0])
    y_rotation=rotation_matrix(axes[1], angles[1])
    z_rotation=rotation_matrix(axes[2], angles[2])
    R = np.matmul(z_rotation, x_rotation, y_rotation)

    radius_point = np.matmul(R,axes[0])*row['radius'] + center
    axis_point = np.matmul(R,axes[2])*row['half_length']*0.5 + center
    # print(radius_point)
    wrp_df._set_value(i,'radius_point', radius_point) # = 
    wrp_df._set_value(i,'axis_point', axis_point)

wrp_df.head()

,body,name,rotation,translation,radius,half_length,radius_point,axis_point
0,pelvis,Gmax1_at_pelvis_r,"[-0.6, 0.45, 0.0]","[-77.0, -99.0, 61.0]",40.0,75.0,"[-37.0, -99.0, 61.0]","[-77.0, -77.82590724768617, 91.95008555911294]"
1,pelvis,Gmax2_at_pelvis_r,"[-0.75, 0.39, 0.0]","[-80.0, -83.0, 68.0]",40.0,50.0,"[-40.0, -83.0, 68.0]","[-80.0, -65.95903099941664, 86.29222172184552]"
2,pelvis,Gmax3_at_pelvis_r,"[-0.1, 0.0, 0.0]","[-83.0, -88.0, 68.0]",40.0,50.0,"[-43.0, -88.0, 68.0]","[-83.0, -85.5041645838293, 92.87510413195065]"
3,pelvis,Gmax1_at_pelvis_l,"[0.6, -0.45, 0.0]","[-77.0, -99.0, -61.0]",40.0,75.0,"[-37.0, -99.0, -61.0]","[-77.0, -120.17409275231383, -30.04991444088707]"
4,pelvis,Gmax2_at_pelvis_l,"[0.75, -0.39, 0.0]","[-80.0, -83.0, -68.0]",40.0,50.0,"[-40.0, -83.0, -68.0]","[-80.0, -100.04096900058336, -49.70777827815448]"


In [81]:
# create a list of bodies and split data by them
body_lst = wrp_df['body'].unique()
for body in body_lst:
    temp = []
    temp.append((wrp_df[wrp_df['body']==body])[['name', 'translation','radius_point', 'axis_point']])
    globals()[f'{body}_wrp'] = pd.concat(temp)
pelvis_wrp.head()

,name,translation,radius_point,axis_point
0,Gmax1_at_pelvis_r,"[-77.0, -99.0, 61.0]","[-37.0, -99.0, 61.0]","[-77.0, -77.82590724768617, 91.95008555911294]"
1,Gmax2_at_pelvis_r,"[-80.0, -83.0, 68.0]","[-40.0, -83.0, 68.0]","[-80.0, -65.95903099941664, 86.29222172184552]"
2,Gmax3_at_pelvis_r,"[-83.0, -88.0, 68.0]","[-43.0, -88.0, 68.0]","[-83.0, -85.5041645838293, 92.87510413195065]"
3,Gmax1_at_pelvis_l,"[-77.0, -99.0, -61.0]","[-37.0, -99.0, -61.0]","[-77.0, -120.17409275231383, -30.04991444088707]"
4,Gmax2_at_pelvis_l,"[-80.0, -83.0, -68.0]","[-40.0, -83.0, -68.0]","[-80.0, -100.04096900058336, -49.70777827815448]"


In [82]:
# create one list of split surfaces
wrapping_surface_markers = [pelvis_wrp, femur_r_wrp, femur_l_wrp, tibia_r_wrp, tibia_l_wrp]

## Import Markers from 3DSlicer file

In [83]:
# create a function for parsing info from a json file
## !! ATTENTION: !! 3DSlicer json separates the position and orientation in two tensors. To obtain the Viewer's orientation one needs to multiply them.
def load_orient_json(path_n_file, orient = True):
    file = open(path_n_file)
    data = json.load(file)
    points = {}
    for item in data['markups'][0]['controlPoints']:
        orient = np.array(item['orientation'])
        orient.shape = (3,3)
        position = np.matmul(orient, np.array(item['position']))
        points[item['label']] = position.tolist()
    return points

path = os.path.abspath("D:\\Docs_ASUS\\WORK\\hip\\MotionStudy\\MotionData\\001\\MRI\\DICOM") #./MRI/003-Anonymous/P03
mri_bone_markers = load_orient_json(os.path.join(path, "orientation.mrk.json"), orient = True)

In [84]:
# check the keys of bone markers and their order
mri_bone_markers.keys()

dict_keys(['ASIS_l', 'ASIS_r', 'PSIS_l', 'PSIS_r', 'knee_l_med', 'knee_l_lat', 'knee_r_med', 'knee_r_lat', 'ankle_r_lat', 'ankle_r_med', 'ankle_l_lat', 'ankle_l_med', 'femur_r_center', 'femur_l_center', 'knee_r_center', 'knee_l_center', 'tibia_l_center', 'tibia_r_center', 'ankle_l_center', 'ankle_r_center', 'patella_r', 'patella_l', 'pelvis_origin', 'ilium_r', 'ilium_l', 'isch_spine_r', 'isch_spine_l', 'isch_tuber_r', 'isch_tuber_l', 'gr_troch_as_r', 'gr_troch_as_l', 'gr_troch_lat_r', 'gr_troch_lat_l', 'isch_infer_r', 'isch_infer_l', 'pub_infer_c', 'patella_sup_r', 'patella_sup_l', 'patella_lat_r', 'patella_med_r', 'patella_lat_l', 'patella_med_l', 'femur_r_midshaft_anter', 'femur_r_midshaft_poster', 'femur_l_midshaft_anter', 'femur_l_midshaft_poster', 'patella_r_anter', 'patella_l_anter', 'fibula_l_as', 'fibula_r_as', 'fibula_l_sup', 'fibula_r_sup', 'tibia_r_midshaft_anter', 'tibia_r_midshaft_poster', 'tibia_l_midshaft_anter', 'tibia_l_midshaft_poster', 'fibula_l_midshaft_anter', 'fib

In [85]:
# View the imported data before any transformations -- check the imported data has correct orientation for right and left sides
import plotly.express as px
import plotly.graph_objects as go

temp = pd.DataFrame.from_dict(mri_bone_markers).T

fig = px.scatter(width=600, height=1000)
fig.add_trace(go.Scatter3d(
x=temp[0], 
y=temp[1], 
z=temp[2],
marker=dict(
    size=4,
    opacity=0.8
),
mode = 'markers + text',
text=temp.index,
textposition="top center"))
fig.update_scenes(aspectmode="data" )
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

## Match OSIM bone markers with Slicer data for TPS function. Add muscle paths.

#### _bone markers_

In [86]:
# match marker names in slicer file to marker names in osim file
# the output file should have the same order as the original slicer import
match_markers = []
no_match_markers = []
for name in mri_bone_markers.keys():
    if name in df_osim_bone_markers.index:
        match_markers.append(name)
    else: no_match_markers.append(name)

osim_bone_markers_df = df_osim_bone_markers.loc[match_markers]

In [87]:
# split osim bone markers by bodies
bodies = osim_bone_markers_df['socket_parent_frame'].unique()
for body in bodies:
    globals()[f'{body}_list'] = []
    globals()[f'{body}_list'].append(osim_bone_markers_df[osim_bone_markers_df['socket_parent_frame']==body])

# create dataframes for bone markers in separate bodies
pelvis_osim = pd.concat(pelvis_list)[['r', 'a', 's']]
femur_r_osim = pd.concat(femur_r_list)[['r', 'a', 's']]
femur_l_osim = pd.concat(femur_l_list)[['r', 'a', 's']]
tibia_r_osim = pd.concat(tibia_r_list)[['r', 'a', 's']]
tibia_l_osim = pd.concat(tibia_l_list)[['r', 'a', 's']]
patella_r_osim = pd.concat(patella_r_list)[['r', 'a', 's']]
patella_l_osim = pd.concat(patella_l_list)[['r', 'a', 's']]
tibia_r_osim.head()

,r,a,s
name,,,
ankle_r_lat,-15.39344,-384.1191,38.67426
ankle_r_med,1.07616,-379.9941,-30.13974
knee_r_center,0.00000,0.0000,0.00000
tibia_r_center,-8.08954,-32.0848,-1.48474
ankle_r_center,-7.15862,-382.0565,4.26730


In [88]:
tibia_r_osim

,r,a,s
name,,,
ankle_r_lat,-15.393440,-384.119100,38.674260
ankle_r_med,1.076160,-379.994100,-30.139740
knee_r_center,0.000000,0.000000,0.000000
tibia_r_center,-8.089540,-32.084800,-1.484740
ankle_r_center,-7.158620,-382.056500,4.267300
fibula_r_as,-5.411190,-63.350428,18.898382
fibula_r_sup,-22.723540,-43.084800,29.005260
tibia_r_midshaft_anter,17.499244,-208.071000,14.208780
tibia_r_midshaft_poster,-3.179098,-208.071000,0.541631


In [89]:
# the support markers are to control the position of joint centres in parent bodies and skin markers
# currently, these are not in the list of MRI markers
not_in_mri_markers = []
for name in df_osim_bone_markers.index:
    if name not in mri_bone_markers.keys():
        not_in_mri_markers.append(name)
    
print(not_in_mri_markers)

['C7', 'RBAK', 'CLAV', 'STRN', 'T10', 'RASI', 'RPSI', 'LPSI', 'LASI', 'PE01', 'PE02', 'PE03', 'RTH1', 'RTH2', 'RTH3', 'RGT', 'RKNE', 'RMKNE', 'LTH1', 'LTH2', 'LTH3', 'LGT', 'LKNE', 'LMKNE', 'RTB1', 'RTB2', 'RTB3', 'RANK', 'RMMA', 'LTB1', 'LTB2', 'LTB3', 'LANK', 'LMMA', 'RHEE', 'RD5M', 'RTOE', 'LHEE', 'LD5M', 'LTOE', 'talus_r_center_in_tibia', 'talus_l_center_in_tibia']


In [90]:
# we only need markers for the centers of talus
osim_support_bone_markers_df = df_osim_bone_markers.loc[not_in_mri_markers[-2:]]

In [91]:
#pelvis_support = osim_support_bone_markers_df[osim_support_bone_markers_df['socket_parent_frame']=='pelvis'][['name', 'r', 'a', 's']]
#femur_r_support = osim_support_bone_markers_df[osim_support_bone_markers_df['socket_parent_frame']=='femur_r'][['name', 'r', 'a', 's']]
#femur_l_support = osim_support_bone_markers_df[osim_support_bone_markers_df['socket_parent_frame']=='femur_l'][['name', 'r', 'a', 's']]
tibia_r_support = osim_support_bone_markers_df[osim_support_bone_markers_df['socket_parent_frame']=='tibia_r'][['r', 'a', 's']]
tibia_l_support = osim_support_bone_markers_df[osim_support_bone_markers_df['socket_parent_frame']=='tibia_l'][['r', 'a', 's']]
#patella_r_support = osim_support_bone_markers_df[osim_support_bone_markers_df['socket_parent_frame']=='patella_r'][['name', 'r', 'a', 's']]
#patella_r_support = osim_support_bone_markers_df[osim_support_bone_markers_df['socket_parent_frame']=='patella_l'][['name', 'r', 'a', 's']]

In [92]:
tibia_l_support 

,r,a,s
name,,,
talus_l_center_in_tibia,-10.0,-400.0,6.140921e-15


#### _muscle paths_

In [93]:
# extract muscles from the OSIM template data
# a function for extracting muscle paths
def extract_data(df, body):
 rows = []
 rows.append(df[df['description']==body])
 df_rows = pd.concat(rows)[['label', 'r', 'a', 's']]
 df_new = df_rows.rename(columns={'label':'name',})
 df_new.set_index('name', inplace=True)
 return df_new

# extract muscles from the OSIM template data
pelvis_osim_mscl = extract_data(df_muscles, 'pelvis')
femur_r_osim_mscl = extract_data(df_muscles, 'femur_r')
femur_l_osim_mscl = extract_data(df_muscles, 'femur_l')
tibia_r_osim_mscl = extract_data(df_muscles, 'tibia_r')
tibia_l_osim_mscl = extract_data(df_muscles, 'tibia_l')
patella_r_osim_mscl = extract_data(df_muscles, 'patella_r')
patella_l_osim_mscl = extract_data(df_muscles, 'patella_l')

femur_l_osim_mscl.head()

,r,a,s
name,,,
addbrev_l-P2,-2.00,-118.00,-24.90
addlong_l-P2,11.26,-239.37,-15.83
addmagDist_l-P2,11.25,-262.50,-19.30
addmagIsch_l-P2,4.81,-387.97,32.73
addmagMid_l-P2,2.42,-162.40,-29.22


#### _bone markers from and muscles from the osim together_

In [94]:
# bone markers and muscles together
# bone markers first, then support if present, then muscles
pelvis_osim_mark_mscl = pd.concat([pelvis_osim,  pelvis_osim_mscl])
femur_r_osim_mark_mscl = pd.concat([femur_r_osim, femur_r_osim_mscl])
femur_l_osim_mark_mscl = pd.concat([femur_l_osim, femur_l_osim_mscl])
tibia_r_osim_mark_mscl = pd.concat([tibia_r_osim, tibia_r_support,  tibia_r_osim_mscl])
tibia_l_osim_mark_mscl = pd.concat([tibia_l_osim, tibia_l_support, tibia_l_osim_mscl])
patella_r_osim_mark_mscl = pd.concat([patella_r_osim, patella_r_osim_mscl])
patella_l_osim_mark_mscl = pd.concat([patella_l_osim, patella_l_osim_mscl])

tibia_l_osim_mark_mscl.tail()

,r,a,s
name,,,
tibpost_l-P1,-4.10,-130.40,-10.30
tibpost_l-P2,-16.40,-365.50,17.50
vasint_l-P5,32.57,-63.20,-0.43
vaslat_l-P5,32.54,-63.38,-5.11
vasmed_l-P5,31.90,-63.57,6.78


## Separate 3DSlicer bone markers into bodies

In [95]:
# create a function for splitting the dataframe into bodies
def choose_lms(dict_target, df_template):
    names = df_template.index
    rows = []
    for name in names:
        rows.append({'names':name, 'r' : dict_target[name][0], 'a' : dict_target[name][1], 's' : dict_target[name][2]})
    df = pd.DataFrame.from_dict(rows)
    df.set_index('names', inplace=True)
    return df

pelvis_mri = choose_lms(mri_bone_markers, pelvis_osim)
femur_r_mri = choose_lms(mri_bone_markers, femur_r_osim)
femur_l_mri = choose_lms(mri_bone_markers, femur_l_osim)
tibia_r_mri = choose_lms(mri_bone_markers, tibia_r_osim)
tibia_l_mri = choose_lms(mri_bone_markers, tibia_l_osim)
patella_r_mri = choose_lms(mri_bone_markers, patella_r_osim)
patella_l_mri = choose_lms(mri_bone_markers, patella_l_osim)

pelvis_mri.head()

,r,a,s
names,,,
ASIS_l,-103.208289,19.221344,760.254093
ASIS_r,114.914900,21.652296,759.361274
PSIS_l,-44.464215,-96.939927,802.218384
PSIS_r,58.772861,-93.335182,803.111209
pelvis_origin,5.853309,20.436829,759.807683


In [96]:
# check that the order of markers is the same as in pelvis_osim
pelvis_osim.head()

,r,a,s
name,,,
ASIS_l,0.58789,-3.874000,-126.221000
ASIS_r,0.58789,-3.873502,126.221277
PSIS_l,-137.95000,17.538000,-39.803400
PSIS_r,-137.95000,17.538000,39.803400
pelvis_origin,0.00000,0.000000,0.000000


In [97]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width=600, height=1000)
fig.add_trace(go.Scatter3d(x = pelvis_osim_mark_mscl['r'], y = pelvis_osim_mark_mscl['a'], z=pelvis_osim_mark_mscl['s'], mode = 'markers', name = 'OSIM'))
fig.add_trace(go.Scatter3d(x = pelvis_mri['r'],  y = pelvis_mri['a'], z=pelvis_mri['s'], mode = 'markers', name = 'slicer'))
fig.update_scenes(aspectmode="data" )
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

## TPS markers and muscles

First, we create a transformation such that fits OSIM set of bone markers to the identical list of the slicer bone markers. Then, this transformation is applied to the full list of bone markers and muscle paths. The data is returned in the slicer set of axes.

<div class="alert alert-block alert-info"> <b>NOTE</b> The transformed features are returned in mm. </div>

#### _pelvis_

In [98]:
# create a spline transformation
tps_pelvis = ThinPlateSpline(alpha = 0)
tps_pelvis.fit(pelvis_osim[['r','a','s']].to_numpy(), pelvis_mri[['r','a','s']].to_numpy())

#apply transformation to the full set of bone markers and muscle paths
pelvis_result = tps_pelvis.transform(pelvis_osim_mark_mscl[['r','a','s']].to_numpy())

# save the list of names for bone markers and muscles in the original order
pelvis_mrkrs_mscls_names = list(pelvis_osim_mark_mscl.index)

In [99]:
# Display the result
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width=600, height=1000)
fig.add_trace(go.Scatter3d(x = pelvis_osim_mark_mscl['r'], y = pelvis_osim_mark_mscl['a'], z=pelvis_osim_mark_mscl['s'], mode = 'markers', name = 'OSIM'))
fig.add_trace(go.Scatter3d(x = pelvis_result[:, 0],  y = pelvis_result[:,1], z=pelvis_result[:,2], mode = 'markers', name = 'slicer'))
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

### _femora_

In [100]:
# create a spline transformation
tps_femur_r = ThinPlateSpline(alpha = 0)
tps_femur_r.fit(femur_r_osim[['r','a','s']].to_numpy(), femur_r_mri[['r','a','s']].to_numpy())

# apply transformation to markers and muscles
femur_r_result = tps_femur_r.transform(femur_r_osim_mark_mscl[['r','a','s']].to_numpy())

# save landmark name in the original order
femur_r_mrkrs_mscls_names = list(femur_r_osim_mark_mscl.index)


In [101]:
# create a spline transformation
tps_femur_l = ThinPlateSpline(alpha = 0)
tps_femur_l.fit(femur_l_osim[['r','a','s']].to_numpy(), femur_l_mri[['r','a','s']].to_numpy())

# apply transformation to markers and muscles
femur_l_result = tps_femur_l.transform(femur_l_osim_mark_mscl[['r','a','s']].to_numpy())

# save landmark name in the original order
femur_l_mrkrs_mscls_names = list(femur_l_osim_mark_mscl.index)


### _patellas_

In [102]:
# create a spline transformation
tps_patella_r = ThinPlateSpline(alpha = 0)
tps_patella_r.fit(patella_r_osim[['r','a','s']].to_numpy(), patella_r_mri[['r','a','s']].to_numpy())

# apply transformation to markers and muscles
patella_r_result = tps_patella_r.transform(patella_r_osim_mark_mscl[['r','a','s']].to_numpy())

# save landmark name in the original order
patella_r_mrkrs_mscls_names = list(patella_r_osim_mark_mscl.index)

In [103]:
# create a spline transformation
tps_patella_l = ThinPlateSpline(alpha = 0)
tps_patella_l.fit(patella_l_osim[['r','a','s']].to_numpy(), patella_l_mri[['r','a','s']].to_numpy())

# apply transformation to markers and muscles
patella_l_result = tps_patella_l.transform(patella_l_osim_mark_mscl[['r','a','s']].to_numpy())

# save landmark name in the original order
patella_l_mrkrs_mscls_names = list(patella_l_osim_mark_mscl.index)


### _tibias_

In [104]:
# create a spline transformation
tps_tibia_r = ThinPlateSpline(alpha = 0)
tps_tibia_r.fit(tibia_r_osim[['r','a','s']].to_numpy(), tibia_r_mri[['r','a','s']].to_numpy())

# apply transformation to markers and muscles
tibia_r_result = tps_tibia_r.transform(tibia_r_osim_mark_mscl[['r','a','s']].to_numpy())

# save landmark name in the original order
tibia_r_mrkrs_mscls_names = list(tibia_r_osim_mark_mscl.index)


In [105]:
# create a spline transformation
tps_tibia_l = ThinPlateSpline(alpha = 0)
tps_tibia_l.fit(tibia_l_osim[['r','a','s']].to_numpy(), tibia_l_mri[['r','a','s']].to_numpy())

# apply transformation to markers and muscles
tibia_l_result = tps_tibia_l.transform(tibia_l_osim_mark_mscl[['r','a','s']].to_numpy())

# save landmark name in the original order
tibia_l_mrkrs_mscls_names = list(tibia_l_osim_mark_mscl.index)

In [106]:
# Display the result in slicer
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width=600, height=1000)
fig.add_trace(go.Scatter3d(x = pelvis_result[:, 0],  y = pelvis_result[:,1], z=pelvis_result[:,2], mode = 'markers', name = 'pelvis'))
fig.add_trace(go.Scatter3d(x = femur_r_result[:, 0],  y = femur_r_result[:,1], z=femur_r_result[:,2], mode = 'markers', name = 'femur_r'))
fig.add_trace(go.Scatter3d(x = femur_l_result[:, 0],  y = femur_l_result[:,1], z=femur_l_result[:,2], mode = 'markers', name = 'femur_l'))
fig.add_trace(go.Scatter3d(x = patella_r_result[:, 0],  y = patella_r_result[:,1], z=patella_r_result[:,2], mode = 'markers', name = 'patella_r'))
fig.add_trace(go.Scatter3d(x = patella_l_result[:, 0],  y = patella_l_result[:,1], z=patella_l_result[:,2], mode = 'markers', name = 'patella_l'))
fig.add_trace(go.Scatter3d(x = tibia_r_result[:, 0],  y = tibia_r_result[:,1], z=tibia_r_result[:,2], mode = 'markers', name = 'tibia_r'))
fig.add_trace(go.Scatter3d(x = tibia_l_result[:, 0],  y = tibia_l_result[:,1], z=tibia_l_result[:,2], mode = 'markers', name = 'tibia_l'))
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

## TPS of parameters for muscle wrapping surfaces

Previousely, each wrapping surface was represented as three parameters: its centre = 'translation', axis_point (which is at the location half-size distance from the center along the axis) and radius point (which is at the radius distance from the center, perpendicular to the axis)

Here, these points are treated as all other markers. They are transformed by previousely created TPS superimpositions.

#### _pelvis_

In [107]:
# save name of surfaces in order of appearance in the mother dataframe
pl_n = list(pelvis_wrp['name'])
print(len(pl_n))

10


In [108]:
# create one numpy array
all_pelvis_wrp = np.concatenate([np.array([i for i in pelvis_wrp['radius_point']]), 
                                 np.array([i for i in pelvis_wrp['axis_point']]), 
                                 np.array([i for i in pelvis_wrp['translation']])])

# apply transformation to all points at the same time and then split result by parameter
[pl_r, pl_a, pl_t] = np.split(tps_pelvis.transform(all_pelvis_wrp), [10,20])

#### _femur r_

In [109]:
# save name of surfaces in order of appearance in the mother dataframe
fr_n = list(femur_r_wrp['name'])
print(len(fr_n))

9


In [110]:
# create one numpy array
all_femur_r_wrp = np.concatenate([np.array([i for i in femur_r_wrp['radius_point']]),
                                  np.array([i for i in femur_r_wrp['axis_point']]),
                                  np.array([i for i in femur_r_wrp['translation']])
                                ])
# apply transformation to all points at the same time and then split result by parameter
[fr_r, fr_a, fr_t] = np.split(tps_femur_r.transform(all_femur_r_wrp), [9,18])

#### _femur l_

In [111]:
# save name of surfaces in order of appearance in the mother dataframe
fl_n = list(femur_l_wrp['name'])
print(len(fl_n))

9


In [112]:
all_femur_l_wrp = np.concatenate([np.array([i for i in femur_l_wrp['radius_point']]),
                                  np.array([i for i in femur_l_wrp['axis_point']]),
                                  np.array([i for i in femur_l_wrp['translation']])
                                  ])
[fl_r, fl_a, fl_t] = np.split(tps_femur_l.transform(all_femur_l_wrp), [9,18])

#### _tibia r_

In [113]:
# save name of surfaces in order of appearance in the mother dataframe
tr_n = list(tibia_r_wrp['name'])
print(len(tr_n))

6


In [114]:
all_tibia_r_wrp = np.concatenate([np.array([i for i in tibia_r_wrp['radius_point']]),
                                  np.array([i for i in tibia_r_wrp['axis_point']]),
                                  np.array([i for i in tibia_r_wrp['translation']])
                                  ])

[tr_r, tr_a, tr_t] = np.split(tps_tibia_r.transform(all_tibia_r_wrp), [6,12])

#### _tibia l_

In [115]:
# save name of surfaces in order of appearance in the mother dataframe
tl_n = list(tibia_l_wrp['name'])
print(len(tl_n))

6


In [116]:
all_tibia_l_wrp = np.concatenate([np.array([i for i in tibia_l_wrp['radius_point']]),
                                  np.array([i for i in tibia_l_wrp['axis_point']]),
                                  np.array([i for i in tibia_l_wrp['translation']])
                                  ])
[tl_r, tl_a, tl_t] = np.split(tps_tibia_l.transform(all_tibia_l_wrp), [6,12])

In [117]:
# plot all results
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width=600, height=1000)
fig.add_trace(go.Scatter3d(x = pelvis_result[:, 0],  y = pelvis_result[:,1], z=pelvis_result[:,2], mode = 'markers', name = 'pelvis'))
fig.add_trace(go.Scatter3d(x = femur_r_result[:, 0],  y = femur_r_result[:,1], z=femur_r_result[:,2], mode = 'markers', name = 'femur_r'))
fig.add_trace(go.Scatter3d(x = femur_l_result[:, 0],  y = femur_l_result[:,1], z=femur_l_result[:,2], mode = 'markers', name = 'femur_l'))
fig.add_trace(go.Scatter3d(x = patella_r_result[:, 0],  y = patella_r_result[:,1], z=patella_r_result[:,2], mode = 'markers', name = 'patella_r'))
fig.add_trace(go.Scatter3d(x = patella_l_result[:, 0],  y = patella_l_result[:,1], z=patella_l_result[:,2], mode = 'markers', name = 'patella_l'))
fig.add_trace(go.Scatter3d(x = tibia_r_result[:, 0],  y = tibia_r_result[:,1], z=tibia_r_result[:,2], mode = 'markers', name = 'tibia_r'))
fig.add_trace(go.Scatter3d(x = tibia_l_result[:, 0],  y = tibia_l_result[:,1], z=tibia_l_result[:,2], mode = 'markers', name = 'tibia_l'))

fig.add_trace(go.Scatter3d(x = pl_r[:, 0],  y = pl_r[:,1], z=pl_r[:,2], mode = 'markers', name = 'pelvis_radius'))
fig.add_trace(go.Scatter3d(x = pl_a[:, 0],  y = pl_a[:,1], z=pl_a[:,2], mode = 'markers', name = 'pelvis_axis'))
fig.add_trace(go.Scatter3d(x = pl_t[:, 0],  y = pl_t[:,1], z=pl_t[:,2], mode = 'markers', name = 'pelvis_translation'))

for i, translation in enumerate(tl_t):
    fig.add_trace(go.Scatter3d(x = [translation[0], tl_r[i,0]], y = [translation[1], tl_r[i,1]], z = [translation[2], tl_r[i,2]], marker = dict(size=4, opacity=0.8, color='darkblue')))
for i, translation in enumerate(tl_t):
    fig.add_trace(go.Scatter3d(x = [translation[0], tl_a[i,0]], y = [translation[1], tl_a[i,1]], z = [translation[2], tl_a[i,2]], marker = dict(size=4, opacity=0.8, color='darkred')))

fig.add_trace(go.Scatter3d(x = tl_t[:, 0],  y = tl_t[:,1], z=tl_t[:,2], mode = 'markers', name = 'tibia_l_translation',  marker = dict(size=4, opacity=0.8,color = 'magenta')))


fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

## Import and TPS surfaces

Surfaces are imported from the OSIM template files. It is assumed that markers collected on OSIM model's bones fit the surfaces exactly.

#### pelvis

In [118]:
pl_mesh = pv.read('../Geometry/l_pelvis.vtp')
pl_faces = pl_mesh.faces
pr_mesh = pv.read('../Geometry/r_pelvis.vtp')
pr_faces = pr_mesh.faces
sacrum_mesh = pv.read('../Geometry/sacrum.vtp')
sacrum_faces = sacrum_mesh.faces

In [119]:
pl_pts = tps_pelvis.transform(np.array(pl_mesh.points)*1000)
pr_pts = tps_pelvis.transform(np.array(pr_mesh.points)*1000)
sacrum_pts = tps_pelvis.transform(np.array(sacrum_mesh.points)*1000)

#### femur_r

In [120]:
# import .vtp file from Geometry
fr_mesh = pv.read('../Geometry/r_femur.vtp')

# apply TPS transformation to the sutface, which is first transformed to mm
fr_pts = tps_femur_r.transform(np.array(fr_mesh.points)*1000)
fr_faces = fr_mesh.faces

In [121]:
# plot the original surface with points
fr_mesh_mm = pv.PolyData(fr_mesh.points*1000, fr_mesh.faces)
femur_r_osim_polydata = pv.PolyData(femur_r_osim[['r','a','s']].to_numpy())

plotter = pv.Plotter(window_size=(600, 400))
plotter.add_mesh(fr_mesh_mm)
plotter.add_mesh(femur_r_osim_polydata,  color='blue', point_size=10.0) 
plotter.show(jupyter_backend='trame')

Widget(value="<iframe src='http://localhost:53103/index.html?ui=P_0x1a13b486c40_4&reconnect=auto' style='width…

In [122]:
# plot the transformed result
the_pts_transformed = fr_pts/1000
new_mesh = pv.PolyData(the_pts_transformed, fr_mesh.faces)
#new_mesh.save('model_update/femur_r.stl')

plotter = pv.Plotter(window_size=(600, 400))
#plotter.add_mesh(fr_mesh, color='lightblue', show_edges=True)
plotter.add_mesh(new_mesh, color='red', show_edges=True)
plotter.show(jupyter_backend='trame')

Widget(value="<iframe src='http://localhost:53103/index.html?ui=P_0x1a139412e50_5&reconnect=auto' style='width…

#### femur_l

In [123]:
fl_mesh = pv.read('../Geometry/l_femur.vtp')
fl_pts = tps_femur_l.transform(np.array(fl_mesh.points)*1000)
fl_faces = fl_mesh.faces

#### tibia_r

In [124]:
tr_mesh = pv.read('../Geometry/r_tibia.vtp')
tr_pts = tps_tibia_r.transform(np.array(tr_mesh.points)*1000)
tr_faces = tr_mesh.faces

fib_r_mesh = pv.read('../Geometry/r_fibula.vtp')
fib_r_pts = tps_tibia_r.transform(np.array(fib_r_mesh.points)*1000)
fib_r_faces = fib_r_mesh.faces

In [125]:
# plot old and new surface in meters
the_pts_transformed = tr_pts /1000
new_mesh = pv.PolyData(the_pts_transformed, tr_faces)
plotter = pv.Plotter(window_size=(600, 400))
plotter.add_mesh(tr_mesh, color='lightblue', show_edges=True)
plotter.add_mesh(new_mesh, color='red', show_edges=True)
plotter.show(jupyter_backend='trame')

Widget(value="<iframe src='http://localhost:53103/index.html?ui=P_0x1a13c8374f0_6&reconnect=auto' style='width…

#### tibia_l

In [126]:
tl_mesh = pv.read('../Geometry/l_tibia.vtp')
tl_pts = tps_tibia_l.transform(np.array(tl_mesh.points)*1000)
tl_faces = tl_mesh.faces

fib_l_mesh = pv.read('../Geometry/l_fibula.vtp')
fib_l_pts = tps_tibia_l.transform(np.array(fib_l_mesh.points)*1000)
fib_l_faces = fib_l_mesh.faces

In [127]:
# plot old and new surface in meters
the_pts_transformed = tl_pts/1000
new_mesh = pv.PolyData(the_pts_transformed, tl_faces)
plotter = pv.Plotter(window_size=(600, 400))
plotter.add_mesh(tl_mesh, color='lightblue', show_edges=True)
plotter.add_mesh(new_mesh, color='red', show_edges=True)
plotter.show(jupyter_backend='trame')

Widget(value="<iframe src='http://localhost:53103/index.html?ui=P_0x1a13c814640_7&reconnect=auto' style='width…

#### patella_r

In [128]:
ptlr_mesh = pv.read('../Geometry/r_patella.vtp')
ptlr_pts = tps_patella_r.transform(np.array(ptlr_mesh.points)*1000)
ptlr_faces = ptlr_mesh.faces

#### patella_l

In [129]:
ptll_mesh = pv.read('../Geometry/l_patella.vtp')
ptll_pts = tps_patella_l.transform(np.array(ptll_mesh.points)*1000)
ptll_faces = ptll_mesh.faces

In [130]:
# display patella result together with point names
import plotly.graph_objects as go
import plotly.express as px

fig = px.scatter(width=600, height=600)

fig.add_trace(go.Scatter3d(x=ptlr_pts[:,0], y=ptlr_pts[:,1], z=ptlr_pts[:,2], 
                           marker=dict(size=4, opacity=0.8), name = 'surface points', mode = 'markers'))

fig.add_trace(go.Scatter3d(x=patella_r_result[:,0], y=patella_r_result[:,1],  z=patella_r_result[:,2], 
                           marker=dict(size=4,opacity=0.8),
                            mode = 'markers + text',
                            text=patella_r_osim_mark_mscl.index,
                            textposition="top center", name = 'transformed markers'))

fig.update_scenes(aspectmode="data" )
fig.show()

## Rotate bodies to child 

In 3dSlicer, axes are directed as r-a-s _left_to_right -- posterior_to_anterior -- inferior_to_superior_. This is also evident from plots above.
We need to rotate 3dSlicer axes to OSIM, were x-y-z are equivalent to _posterior_to_anterior -- inferior_to_superior -- left_to_right_. 

This can be achieved by first rotating 3dSlicer coordinates 90° anticlockwise around x, and then 90° anticlockwise around y.

Each body also needs to be centered at the joint center.

In [131]:
rotation_matrix([1,0,0], -np.pi/2)

array([[ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0., -1.,  0.]])

In [132]:
rotation_matrix([0,1,0], -np.pi/2)

array([[ 0.,  0., -1.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.]])

In [133]:
ROT = np.matmul(rotation_matrix([0,1,0], -np.pi/2), rotation_matrix([1,0,0], -np.pi/2))
ROT

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [134]:
rotate_pelvis = np.matmul(pelvis_result - np.mean(pelvis_result, axis = 0), ROT.T)

In [135]:
# Display the result of rotation for pelvis -- it is clear that the chosen rotation matiry should work correctly.
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width=600, height=600)
fig.add_trace(go.Scatter3d(x = rotate_pelvis[:, 0],  y = rotate_pelvis[:,1], z=rotate_pelvis[:,2], mode = 'markers', name = 'pelvis', marker=dict(size=5,opacity=0.8)))
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

In [136]:
def plot_transformed(points_file, order_file, mrkr_mscl_names):
   fig = px.scatter()
   for key in order_file.keys():
      if key == 'mrkrs_mscls':
         fig.add_trace(go.Scatter3d(
            x=points_file[order_file['mrkrs_mscls'],0], 
            y=points_file[order_file['mrkrs_mscls'],1], 
            z=points_file[order_file['mrkrs_mscls'],2],
            marker=dict(
               size=4,
               opacity=0.8
            ),
            mode = 'markers + text',
            text=mrkr_mscl_names,
            textposition="top center"))
      else:
         fig.add_trace(go.Scatter3d(
            x=points_file[order_file[key],0], 
            y=points_file[order_file[key],1], 
            z=points_file[order_file[key],2],
            marker=dict(
               size=4,
               opacity=0.8
            ),
            mode = 'markers',
         ))

   fig.update_scenes(aspectmode="data" )
   fig.show()

The next challenge is to straighten up orientation of bodies so that their axes correspond with the OSIM prescriptions.

To do so, we first need to decide on how body set of axes should be created and then rotate it to the desired orientation.

In [137]:
# first, assume that the desired orientation correspond with the axes swap from 3dSlicer to OSIM.
axes_swap = np.array([[0,0,1], [1,0,0], [0,1,0]])
axes_osim = axes_swap*50

In [138]:
# at this moment, transformed results exist as np.arrays by bodies
print(pelvis_mrkrs_mscls_names)
print(len(pelvis_mrkrs_mscls_names))
print(pelvis_result.shape)

['ASIS_l', 'ASIS_r', 'PSIS_l', 'PSIS_r', 'pelvis_origin', 'ilium_r', 'ilium_l', 'isch_spine_r', 'isch_spine_l', 'isch_tuber_r', 'isch_tuber_l', 'isch_infer_r', 'isch_infer_l', 'pub_infer_c', 'femur_r_center_in_pelvis', 'femur_l_center_in_pelvis', 'torso_origin_in_pelvis', 'addbrev_r-P1', 'addlong_r-P1', 'addmagDist_r-P1', 'addmagIsch_r-P1', 'addmagMid_r-P1', 'addmagProx_r-P1', 'bflh_r-P1', 'glmax1_r-P1', 'glmax1_r-P2', 'glmax2_r-P1', 'glmax2_r-P2', 'glmax3_r-P1', 'glmax3_r-P2', 'glmed1_r-P1', 'glmed2_r-P1', 'glmed3_r-P1', 'glmin1_r-P1', 'glmin2_r-P1', 'glmin3_r-P1', 'grac_r-P1', 'iliacus_r-P1', 'iliacus_r-P2', 'piri_r-P1', 'piri_r-P2', 'psoas_r-P1', 'psoas_r-P2', 'recfem_r-P1', 'sart_r-P1', 'semimem_r-P1', 'semiten_r-P1', 'tfl_r-P1', 'addbrev_l-P1', 'addlong_l-P1', 'addmagDist_l-P1', 'addmagIsch_l-P1', 'addmagMid_l-P1', 'addmagProx_l-P1', 'bflh_l-P1', 'glmax1_l-P1', 'glmax1_l-P2', 'glmax2_l-P1', 'glmax2_l-P2', 'glmax3_l-P1', 'glmax3_l-P2', 'glmed1_l-P1', 'glmed2_l-P1', 'glmed3_l-P1', '

In [139]:
# at the same time, mri_bone_markers is a dictionary, whose order should correspond to the order of markers split by bodies
# check names of lms, create a table for all and extract eigenvalues to determine the major vertical axis
# li = []
# for key in mri_bone_markers.keys():
#     li.append(mri_bone_markers[key])
# eval, evec = np.linalg.eigh(np.array(li).T@np.array(li), UPLO='L')

#### _pelvis_

In [140]:
mri_bone_markers.keys()

dict_keys(['ASIS_l', 'ASIS_r', 'PSIS_l', 'PSIS_r', 'knee_l_med', 'knee_l_lat', 'knee_r_med', 'knee_r_lat', 'ankle_r_lat', 'ankle_r_med', 'ankle_l_lat', 'ankle_l_med', 'femur_r_center', 'femur_l_center', 'knee_r_center', 'knee_l_center', 'tibia_l_center', 'tibia_r_center', 'ankle_l_center', 'ankle_r_center', 'patella_r', 'patella_l', 'pelvis_origin', 'ilium_r', 'ilium_l', 'isch_spine_r', 'isch_spine_l', 'isch_tuber_r', 'isch_tuber_l', 'gr_troch_as_r', 'gr_troch_as_l', 'gr_troch_lat_r', 'gr_troch_lat_l', 'isch_infer_r', 'isch_infer_l', 'pub_infer_c', 'patella_sup_r', 'patella_sup_l', 'patella_lat_r', 'patella_med_r', 'patella_lat_l', 'patella_med_l', 'femur_r_midshaft_anter', 'femur_r_midshaft_poster', 'femur_l_midshaft_anter', 'femur_l_midshaft_poster', 'patella_r_anter', 'patella_l_anter', 'fibula_l_as', 'fibula_r_as', 'fibula_l_sup', 'fibula_r_sup', 'tibia_r_midshaft_anter', 'tibia_r_midshaft_poster', 'tibia_l_midshaft_anter', 'tibia_l_midshaft_poster', 'fibula_l_midshaft_anter', 'fib

Pelvis axes are taken: infero-superior to coinside with the last (the largest) eigenvector from above; postero-anterior is the axis between PSIS midpoint and ASIS midpoint (which is also 'pelvis_origin'). The left-to right axis is a cross product of the first two axes.

In [146]:
# rotation matrix
p1 = np.array(mri_bone_markers['ASIS_r'])
p2 =(np.array(mri_bone_markers['PSIS_l']) + np.array(mri_bone_markers['PSIS_r']))*0.5
p3 = np.array(mri_bone_markers['ASIS_l'])
p4 = mri_bone_markers['pelvis_origin']
pelvis_location = p4

pelvis_PA = (p4-p2)/np.linalg.norm(p4-p2) # PA stands for posterior -> anterior
pelvis_LR = (p1-p3)/np.linalg.norm(p1-p3)
pelvis_IS = np.cross(pelvis_LR, pelvis_PA)# LR stand s for left -> right]

axes_image = (np.array((pelvis_LR, pelvis_PA, pelvis_IS)) * 50) + pelvis_location 

pelvis_R, pelvis_t, pelvis_center_osim, pelvis_center_image = axes_rotation(axes_osim, axes_image) # mri and osim are 3x3 matrices - second is transferred to the first

In [147]:
# pelvis rotation should roughly correspond to ROT matrix that was found earlier, which is indeed true:
pelvis_R

array([[-1.12129883e-02,  9.37629306e-01, -3.47455828e-01],
       [-2.54395917e-04,  3.47474987e-01,  9.37689217e-01],
       [ 9.99937100e-01,  1.06026896e-02, -3.65770390e-03]])

In [148]:
# here, we assemble all pelvis data for transformation and keep record of the order
obj_order = ['mrkrs_mscls', 'wrp_rad', 'wrp_ax', 'wrp_transl', 'pelv_l_pts', 'pelv_r_pts', 'sacr_pts']
pelvis_lm_order = {}
n = 0
for i, num in enumerate([pelvis_result.shape[0], pl_r.shape[0], pl_a.shape[0], pl_t.shape[0], pl_pts.shape[0], pr_pts.shape[0], sacrum_pts.shape[0]]):
    pelvis_lm_order[obj_order[i]] = list(range(n, n+num))
    n += num

pelvis_to_transform = np.concatenate([pelvis_result, pl_r, pl_a, pl_t, pl_pts, pr_pts, sacrum_pts])

In [149]:
# rotate and shift origin to 0,0,0
pelvis_rotated = apply_rotate_zero_origin(pelvis_to_transform, pelvis_R)
pelvis_origin = np.mean(pelvis_rotated[:2], axis = 0) # mean of ASIS potins
pelvis_transformed = pelvis_rotated - pelvis_origin

In [150]:
# Display the result of rotation for pelvis -- it is clear that the chosen rotation matiry should work correctly.
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width=600, height=600)
fig.add_trace(go.Scatter3d(x = pelvis_transformed[:, 0],  y = pelvis_transformed[:,1], z=pelvis_transformed[:,2], mode = 'markers', name = 'pelvis', marker=dict(size=5,opacity=0.8)))
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

*pelvis_transformed is in the desired orientation*

In [152]:
# scale surface for OpenSim and save
l_pelvis_transformed = pv.PolyData(pelvis_transformed[pelvis_lm_order['pelv_l_pts']]/1000, pl_faces)
r_pelvis_transformed = pv.PolyData(pelvis_transformed[pelvis_lm_order['pelv_r_pts']]/1000, pr_faces)
sacrum_transformed = pv.PolyData(pelvis_transformed[pelvis_lm_order['sacr_pts']]/1000, sacrum_faces)

In [154]:
import shutil
output_path = os.path.join('model_update', 'tps_warping_results')
if not os.path.exists(output_path):
    print(output_path)
    os.makedirs(output_path)

l_pelvis_transformed.save('model_update/tps_warping_results/l_pelvis.stl')
r_pelvis_transformed.save('model_update/tps_warping_results/r_pelvis.stl')
sacrum_transformed.save('model_update/tps_warping_results/sacrum.stl')

model_update\tps_warping_results


In [155]:
# scale markers and muscles, create a dataframe for sorting and saving
body = 'pelvis'
pelvis_transformed_mrkr_mscl = {'body':[], 'name':[], 'location':[]}
for i, n in enumerate(pelvis_mrkrs_mscls_names):
    pelvis_transformed_mrkr_mscl['body'].append('pelvis'),
    pelvis_transformed_mrkr_mscl['name'].append(n),
    pelvis_transformed_mrkr_mscl['location'].append(pelvis_transformed[pelvis_lm_order['mrkrs_mscls']][i]/1000)
pelvis_transformed_mrkr_mscl_df = pd.DataFrame(pelvis_transformed_mrkr_mscl)

In [156]:
pelvis_transformed_mrkr_mscl_df.head()

,body,name,location
0,pelvis,ASIS_l,"[-7.186732357303782e-05, 2.3990873717778525e-0..."
1,pelvis,ASIS_r,"[7.186732357304493e-05, -2.399087371777142e-05..."
2,pelvis,PSIS_l,"[-0.12422751337718722, -0.0010046268030227878,..."
3,pelvis,PSIS_r,"[-0.12231541182224469, 0.0010588616252933248, ..."
4,pelvis,pelvis_origin,"[8.170444289135048e-09, 2.6178743510740786e-09..."


#### *femur_r* and *patella_r*

Femur axes are set as following: infero-superior axis coincides with the vector from the center of  the knee to the center of the femur head; the postero-anterior axis is set perpendicular to the plane created between the center of the femur head, lateral and medial points of the knee. Finally, the left-to-right vector is a cross product of the two above.

In [157]:
# origin
for i, n in enumerate(femur_r_mrkrs_mscls_names):
    if n == 'femur_r_center':
        femur_r_location = i

# rotation matrix
points = ['femur_r_center', 'knee_r_med', 'knee_r_lat']

femur_r_pa = find_axis(mri_bone_markers, points)
femur_r_is = (np.array(mri_bone_markers['femur_r_center']) - np.array(mri_bone_markers['knee_r_center']))/np.linalg.norm(np.array(mri_bone_markers['femur_r_center']) - np.array(mri_bone_markers['knee_r_center']))
femur_r_lr = np.cross(femur_r_pa, femur_r_is)/np.linalg.norm(np.cross(femur_r_pa, femur_r_is))

femur_r_axes = np.array((femur_r_lr, femur_r_pa, femur_r_is)) * 50 + mri_bone_markers['femur_r_center']

femur_r_R, femur_r_t, femur_r_center_osim, femur_r_center_mri = axes_rotation(axes_osim, femur_r_axes)

the determinant is less than zero, recalculate r


In [158]:
# femur_r is rotated more than pelvis relative the desired frame, hence the rotation matrix is more different from the ideal
femur_r_R

array([[ 0.10219407,  0.9935146 ,  0.04985089],
       [ 0.02157944, -0.05231557,  0.99839742],
       [ 0.99453039, -0.10095454, -0.02678583]])

In [159]:
# femur_r: assemble data for transformation and keep record of the order
obj_order = ['mrkrs_mscls', 'wrp_rad', 'wrp_ax', 'wrp_transl', 'surf_pts']
femur_r_lm_order = {}
n = 0
for i, num in enumerate([femur_r_result.shape[0], fr_r.shape[0], fr_a.shape[0], fr_t.shape[0], fr_pts.shape[0]]):
    femur_r_lm_order[obj_order[i]] = list(range(n, n+num))
    n += num

femur_r_to_transform = np.concatenate([femur_r_result, fr_r, fr_a, fr_t, fr_pts])
print(femur_r_to_transform.shape)

(527, 3)


In [160]:
fig = px.scatter(width=800, height=800)
fig.add_trace(go.Scatter3d(x = femur_r_result[:, 0],  y = femur_r_result[:,1], z=femur_r_result[:,2], mode = 'markers', name = 'femur_r_to_transform', marker=dict(size=5,opacity=0.8)))
fig.add_trace(go.Scatter3d(x = fr_r[:, 0],  y = fr_r[:,1], z=fr_r[:,2], mode = 'markers', name = 'fr_r', marker=dict(size=5,opacity=0.8)))
fig.add_trace(go.Scatter3d(x = fr_a[:, 0],  y = fr_a[:,1], z=fr_a[:,2], mode = 'markers', name = 'fr_a', marker=dict(size=5,opacity=0.8)))
fig.add_trace(go.Scatter3d(x = fr_t[:, 0],  y = fr_t[:,1], z=fr_t[:,2], mode = 'markers', name = 'fr_t', marker=dict(size=5,opacity=0.8)))
fig.add_trace(go.Scatter3d(x = fr_pts[:, 0],  y = fr_pts[:,1], z=fr_pts[:,2], mode = 'markers', name = 'fr_r', marker=dict(size=5,opacity=0.8)))
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

We transform femur and patella together to avoid uncertainties in location of the patella axes.

In [161]:
# patella_r: assemble data for transformation and keep record of the order
for i, n in enumerate(patella_r_mrkrs_mscls_names):
    if n == 'patella_r':
        patella_r_location = i

obj_order = ['mrkrs_mscls','surf_pts']
patella_r_lm_order = {}
n = 0
for i, num in enumerate([patella_r_result.shape[0], ptlr_pts.shape[0]]):
    patella_r_lm_order[obj_order[i]] = list(range(n, n+num))
    n += num

patella_r_to_transform = np.concatenate([patella_r_result, ptlr_pts])
print(patella_r_to_transform.shape)

(86, 3)


In [162]:
femur_r_patella_r_rotated = apply_rotate_zero_origin(np.concatenate([femur_r_to_transform, patella_r_to_transform]), femur_r_R)
femur_r_patella_r_transformed = femur_r_patella_r_rotated - femur_r_patella_r_rotated[femur_r_location]
[femur_r_transformed, patella_r_rotated] = np.split(femur_r_patella_r_transformed, [femur_r_to_transform.shape[0]])

In [163]:
fig = px.scatter(width=800, height=800)
fig.add_trace(go.Scatter3d(x = femur_r_transformed[:, 0],  y = femur_r_transformed[:,1], z=femur_r_transformed[:,2], mode = 'markers', name = 'femur_r_transformed', marker=dict(size=5,opacity=0.8)))
fig.add_trace(go.Scatter3d(x = patella_r_rotated[:, 0],  y = patella_r_rotated[:,1], z=patella_r_rotated[:,2], mode = 'markers', name = 'patella_r_transformed_1', marker=dict(size=5,opacity=0.8)))
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

In [164]:
patella_r_transformed = patella_r_rotated - patella_r_rotated[patella_r_location]

In [165]:
fig = px.scatter(width=600, height=600)
fig.add_trace(go.Scatter3d(x = patella_r_transformed[:, 0],  y = patella_r_transformed[:,1], z=patella_r_transformed[:,2], mode = 'markers', name = 'patella_r_transformed_1', marker=dict(size=5,opacity=0.8)))
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

In [166]:
# femur_r: scale surface for OpenSim and save
femur_pts_transformed = femur_r_transformed[femur_r_lm_order['surf_pts']]/1000
new_mesh = pv.PolyData(femur_pts_transformed, fr_faces)
new_mesh.save('model_update/tps_warping_results/femur_r.stl')

# femur_r: scale markers and muscles, create a dataframe for sorting and saving
body = 'femur_r'
femur_r_transformed_mrkr_mscl = {'body':[], 'name':[], 'location':[]}
for i, n in enumerate(femur_r_mrkrs_mscls_names):
    femur_r_transformed_mrkr_mscl['body'].append('femur_r'),
    femur_r_transformed_mrkr_mscl['name'].append(n),
    femur_r_transformed_mrkr_mscl['location'].append(femur_r_transformed[femur_r_lm_order['mrkrs_mscls']][i]/1000)
femur_r_transformed_mrkr_mscl_df = pd.DataFrame(femur_r_transformed_mrkr_mscl)

In [167]:
# patella_r: scale surface for OpenSim and save
patella_pts_transformed = patella_r_transformed[patella_r_lm_order['surf_pts']]/1000
new_mesh = pv.PolyData(patella_pts_transformed, ptlr_faces)
new_mesh.save('model_update/tps_warping_results/patella_r.stl')

# patella_r: scale markers and muscles, create a dataframe for sorting and saving
body = 'patella_r'
patella_r_transformed_mrkr_mscl = {'body':[], 'name':[], 'location':[]}
for i, n in enumerate(patella_r_mrkrs_mscls_names):
    patella_r_transformed_mrkr_mscl['body'].append('patella_r'),
    patella_r_transformed_mrkr_mscl['name'].append(n),
    patella_r_transformed_mrkr_mscl['location'].append(patella_r_transformed[patella_r_lm_order['mrkrs_mscls']][i]/1000)
patella_r_transformed_mrkr_mscl_df = pd.DataFrame(patella_r_transformed_mrkr_mscl)

#### *femur_l* and *patella_l*

In [168]:
# origin
for i, n in enumerate(femur_l_mrkrs_mscls_names):
    if n == 'femur_l_center':
        femur_l_location = i

# rotation matrix
points = ['femur_l_center', 'knee_l_lat', 'knee_l_med']

femur_l_pa = find_axis(mri_bone_markers, points)
femur_l_is = (np.array(mri_bone_markers['femur_l_center']) - np.array(mri_bone_markers['knee_l_center']))/np.linalg.norm(np.array(mri_bone_markers['femur_l_center']) - np.array(mri_bone_markers['knee_l_center']))
femur_l_lr = np.cross(femur_l_pa, femur_r_is)/np.linalg.norm(np.cross(femur_l_pa, femur_l_is))

femur_l_axes = np.array((femur_l_lr, femur_l_pa, femur_l_is)) * 50 + mri_bone_markers['femur_l_center']

femur_l_R, femur_l_t, femur_l_center_osim, femur_l_center_mri = axes_rotation(axes_osim, femur_l_axes)

the determinant is less than zero, recalculate r


In [169]:
# femur_l: assemble data for transformation and keep record of the order
obj_order = ['mrkrs_mscls', 'wrp_rad', 'wrp_ax', 'wrp_transl', 'surf_pts']
femur_l_lm_order = {}
n = 0
for i, num in enumerate([femur_l_result.shape[0], fl_r.shape[0], fl_a.shape[0], fl_t.shape[0], fl_pts.shape[0]]):
    femur_l_lm_order[obj_order[i]] = list(range(n, n+num))
    n += num

femur_l_to_transform = np.concatenate([femur_l_result, fl_r, fl_a, fl_t, fl_pts])
print(femur_l_to_transform.shape)

(527, 3)


In [170]:
fig = px.scatter(width=800, height=800)
fig.add_trace(go.Scatter3d(x = femur_l_result[:, 0],  y = femur_l_result[:,1], z=femur_l_result[:,2], mode = 'markers', name = 'femur_l_to_transform', marker=dict(size=5,opacity=0.8)))
fig.add_trace(go.Scatter3d(x = fl_r[:, 0],  y = fl_r[:,1], z=fl_r[:,2], mode = 'markers', name = 'fr_r', marker=dict(size=5,opacity=0.8)))
fig.add_trace(go.Scatter3d(x = fl_a[:, 0],  y = fl_a[:,1], z=fl_a[:,2], mode = 'markers', name = 'fr_a', marker=dict(size=5,opacity=0.8)))
fig.add_trace(go.Scatter3d(x = fl_t[:, 0],  y = fl_t[:,1], z=fl_t[:,2], mode = 'markers', name = 'fr_t', marker=dict(size=5,opacity=0.8)))
fig.add_trace(go.Scatter3d(x = fl_pts[:, 0],  y = fl_pts[:,1], z=fl_pts[:,2], mode = 'markers', name = 'fr_r', marker=dict(size=5,opacity=0.8)))
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

In [171]:
# patella_l: origin
for i, n in enumerate(patella_l_mrkrs_mscls_names):
    if n == 'patella_l':
        patella_l_location =  i

# patella_l: assemble data for transformation and keep record of the order
obj_order = ['mrkrs_mscls','surf_pts']
patella_l_lm_order = {}
n = 0
for i, num in enumerate([patella_l_result.shape[0], ptll_pts.shape[0]]):
    patella_l_lm_order[obj_order[i]] = list(range(n, n+num))
    n += num

patella_l_to_transform = np.concatenate([patella_l_result, ptll_pts]) 
print(patella_l_to_transform.shape)

(86, 3)


In [172]:
# rotate and shift origin to femur head
femur_l_patella_l_rotated = apply_rotate_zero_origin(np.concatenate([femur_l_to_transform,patella_l_to_transform]), femur_l_R)
femur_l_patella_l_transformed= femur_l_patella_l_rotated - femur_l_patella_l_rotated[femur_l_location]
[femur_l_transformed, patella_l_rotated] = np.split(femur_l_patella_l_transformed, [femur_l_to_transform.shape[0]])
patella_l_transformed = patella_l_rotated - patella_l_rotated[patella_l_location]

In [173]:
fig = px.scatter()
fig.add_trace(go.Scatter3d(x=femur_l_transformed[:,0], y=femur_l_transformed[:,1], z=femur_l_transformed[:,2], 
                           marker=dict(size=4, opacity=0.8), mode = 'markers', name = 'femur_l'))

fig.add_trace(go.Scatter3d(x=patella_l_rotated[:,0], y=patella_l_rotated[:,1], z=patella_l_rotated[:,2],
                           marker=dict(size=4, opacity=0.8), mode = 'markers', name = 'patella_l'))

fig.update_scenes(aspectmode="data" )
fig.show()

In [174]:
fig = px.scatter()

fig.add_trace(go.Scatter3d(x=patella_l_transformed[:,0], y=patella_l_transformed[:,1], z=patella_l_transformed[:,2],
                           marker=dict(size=4, opacity=0.8), mode = 'markers', name = 'patella_l'))

fig.update_scenes(aspectmode="data" )
fig.show()

In [175]:
# femur_l : scale surface for OpenSim and save
femur_pts_transformed = femur_l_transformed[femur_l_lm_order['surf_pts']]/1000
new_mesh = pv.PolyData(femur_pts_transformed, fl_faces)
new_mesh.save('model_update/tps_warping_results/femur_l.stl')

# femur_l : scale markers and muscles, create a dataframe for sorting and saving
body = 'femur_l'
femur_l_transformed_mrkr_mscl = {'body':[], 'name':[], 'location':[]}
for i, n in enumerate(femur_l_mrkrs_mscls_names):
    femur_l_transformed_mrkr_mscl['body'].append('femur_l'),
    femur_l_transformed_mrkr_mscl['name'].append(n),
    femur_l_transformed_mrkr_mscl['location'].append(femur_l_transformed[femur_l_lm_order['mrkrs_mscls']][i]/1000)
femur_l_transformed_mrkr_mscl_df = pd.DataFrame(femur_l_transformed_mrkr_mscl)

In [176]:
# patella_l : scale surface for OpenSim and save
patella_l_pts_transformed = patella_l_transformed[patella_l_lm_order['surf_pts']]/1000
new_mesh = pv.PolyData(patella_l_pts_transformed, ptll_faces)
new_mesh.save('model_update/tps_warping_results/patella_l.stl')

# scale markers and muscles, create a dataframe for sorting and saving
body = 'patella_l'
patella_l_transformed_mrkr_mscl = {'body':[], 'name':[], 'location':[]}
for i, n in enumerate(patella_l_mrkrs_mscls_names):
    patella_l_transformed_mrkr_mscl['body'].append('patella_l'),
    patella_l_transformed_mrkr_mscl['name'].append(n),
    patella_l_transformed_mrkr_mscl['location'].append(patella_l_transformed[patella_l_lm_order['mrkrs_mscls']][i]/1000)
patella_l_transformed_mrkr_mscl_df = pd.DataFrame(patella_l_transformed_mrkr_mscl)

#### *tibia_r*

Tibia axes are defined so that the postero-anterior axis is perpendicular to the plane through lateral and medial tibial points around tibial plateau and through the ankle center. Then, inferosuperior axis coinsides with the vector from ankle center to knee ceter, and the left-right axis is the cross-product of the two above.

In [177]:
# tibia_r origin
for i, n in enumerate(tibia_r_mrkrs_mscls_names):
    if n == 'knee_r_center':
        tibia_r_location =  i

# rotation matrix
points = ['tibia_r_med', 'ankle_r_center', 'tibia_r_lat']
tibia_r_pa = find_axis(mri_bone_markers, points)

tibia_r_is = (np.array(mri_bone_markers['knee_r_center']) - np.array(mri_bone_markers['ankle_r_center']))/np.linalg.norm(np.array(mri_bone_markers['ankle_r_center']) - np.array(mri_bone_markers['knee_r_center']))
tibia_r_lr = (np.cross(tibia_r_pa, tibia_r_is))/np.linalg.norm(np.cross(tibia_r_pa, tibia_r_is))

tibia_r_axes = np.array((tibia_r_lr, tibia_r_pa, tibia_r_is)) * 50 + mri_bone_markers['knee_r_center']

tibia_r_R, tibia_r_t, tibia_r_center_osim, tibia_r_center_mri = axes_rotation(axes_osim, tibia_r_axes)
tibia_r_R

the determinant is less than zero, recalculate r


array([[ 1.22228530e-01,  9.92483524e-01,  6.05323533e-03],
       [-4.37511385e-02, -7.05100113e-04,  9.99042212e-01],
       [ 9.91537203e-01, -1.22376297e-01,  4.33361008e-02]])

Out of all bones so far, tibia is rotated the most. The rotation matirx is significantly different from the ideal.

In [178]:
# assemble data for transformation and keep record of the order
obj_order = ['mrkrs_mscls', 'wrp_rad', 'wrp_ax', 'wrp_transl', 'tib_surf_pts', 'fib_surf_pts']
tibia_r_lm_order = {}
n = 0
for i, num in enumerate([tibia_r_result.shape[0], tr_r.shape[0], tr_a.shape[0], tr_t.shape[0], tr_pts.shape[0], fib_r_pts.shape[0]]):
    tibia_r_lm_order[obj_order[i]] = list(range(n, n+num))
    n += num

tibia_r_to_transform = np.concatenate([tibia_r_result, tr_r, tr_a, tr_t, tr_pts, fib_r_pts])

In [179]:
# rotate and shift origin to 0,0,0
tibia_r_rotated = apply_rotate_zero_origin(tibia_r_to_transform, tibia_r_R)
tibia_r_transformed = tibia_r_rotated - tibia_r_rotated[tibia_r_location]

In [180]:
fig = px.scatter(width=800, height=600)
fig.add_trace(go.Scatter3d(x = tibia_r_transformed[:, 0],  y = tibia_r_transformed[:,1], z=tibia_r_transformed[:,2], mode = 'markers', name = 'tibia_r_transformed_1', marker=dict(size=5,opacity=0.8)))
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

In [181]:
# scale surface for OpenSim and save
tr_pts_transformed = tibia_r_transformed[tibia_r_lm_order['tib_surf_pts']]/1000
tr_new_mesh = pv.PolyData(tr_pts_transformed, tr_faces)
tr_new_mesh.save('model_update/tps_warping_results/tibia_r.stl')

fr_pts_transformed = tibia_r_transformed[tibia_r_lm_order['fib_surf_pts']]/1000
fr_new_mesh = pv.PolyData(fr_pts_transformed, fib_r_faces )
fr_new_mesh.save('model_update/tps_warping_results/fibula_r.stl')

In [182]:
# scale markers and muscles, create a dataframe for sorting and saving
body = 'tibia_r'
tibia_r_transformed_mrkr_mscl = {'body':[], 'name':[], 'location':[]}
for i, n in enumerate(tibia_r_mrkrs_mscls_names):
    tibia_r_transformed_mrkr_mscl['body'].append('tibia_r'),
    tibia_r_transformed_mrkr_mscl['name'].append(n),
    tibia_r_transformed_mrkr_mscl['location'].append(tibia_r_transformed[tibia_r_lm_order['mrkrs_mscls']][i]/1000)
tibia_r_transformed_mrkr_mscl_df = pd.DataFrame(tibia_r_transformed_mrkr_mscl)

#### *tibia_l*

In [183]:
from skspatial.objects import Plane, Points

# tibia_r origin
for i, n in enumerate(tibia_l_mrkrs_mscls_names):
    if n == 'knee_l_center':
        tibia_l_location =  i

# rotation matrix
points = ['tibia_l_lat', 'ankle_l_center', 'tibia_l_med']
tibia_l_pa = find_axis(mri_bone_markers, points)

tibia_l_is = (np.array(mri_bone_markers['knee_l_center'] - np.array(mri_bone_markers['ankle_l_center'])))/np.linalg.norm(np.array(mri_bone_markers['knee_l_center']) - np.array(mri_bone_markers['ankle_l_center']))
tibia_l_lr = (np.cross(tibia_l_pa, tibia_l_is))/np.linalg.norm(np.cross(tibia_l_pa, tibia_l_is))

tibia_l_axes = np.array((tibia_l_lr, tibia_l_pa, tibia_l_is)) * 50 + mri_bone_markers['knee_l_center']

tibia_l_R, tibia_l_t, tibia_l_center_osim, tibia_l_center_mri = axes_rotation(axes_osim, tibia_l_axes)
tibia_l_R

the determinant is less than zero, recalculate r


array([[ 0.01583102,  0.99947204,  0.02837288],
       [ 0.06787334, -0.02938519,  0.99726111],
       [ 0.99756834, -0.0138619 , -0.0683027 ]])

In [451]:
# assemble data for transformation and keep record of the order
obj_order = ['mrkrs_mscls', 'wrp_rad', 'wrp_ax', 'wrp_transl', 'tib_surf_pts', 'fib_surf_pts']
tibia_l_lm_order = {}
n = 0
for i, num in enumerate([tibia_l_result.shape[0], tl_r.shape[0], tl_a.shape[0], tl_t.shape[0], tl_pts.shape[0], fib_l_pts.shape[0]]):
    tibia_l_lm_order[obj_order[i]] = list(range(n, n+num))
    n += num

tibia_l_to_transform = np.concatenate([tibia_l_result, tl_r, tl_a, tl_t, tl_pts, fib_l_pts])

In [452]:
# rotate and shift origin to 0,0,0
tibia_l_rotated = apply_rotate_zero_origin(tibia_l_to_transform, tibia_l_R)
tibia_l_transformed = tibia_l_rotated - tibia_l_rotated[tibia_l_location]

In [453]:
fig = px.scatter(width=800, height=600)
fig.add_trace(go.Scatter3d(x = tibia_l_transformed[:, 0],  y = tibia_l_transformed[:,1], z=tibia_l_transformed[:,2], mode = 'markers', name = 'tibia_l_transformed_1', marker=dict(size=5,opacity=0.8)))
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

In [454]:
# scale surface for OpenSim and save
tl_pts_transformed = tibia_l_transformed[tibia_l_lm_order['tib_surf_pts']]/1000
tl_new_mesh = pv.PolyData(tl_pts_transformed, tl_faces)
tl_new_mesh.save('model_update/tps_warping_results/tibia_l.stl')

fl_pts_transformed = tibia_l_transformed[tibia_l_lm_order['fib_surf_pts']]/1000
fl_new_mesh = pv.PolyData(fl_pts_transformed, fib_l_faces )
fl_new_mesh.save('model_update/tps_warping_results/fibula_l.stl')

In [455]:
plot_transformed(tibia_r_transformed, tibia_r_lm_order, tibia_r_mrkrs_mscls_names)

In [456]:
plot_transformed(tibia_l_transformed, tibia_l_lm_order, tibia_l_mrkrs_mscls_names)

### Display transformed results

In [457]:
#tibia_l_transformed
tibia_l_lm_order.keys()

dict_keys(['mrkrs_mscls', 'wrp_rad', 'wrp_ax', 'wrp_transl', 'tib_surf_pts', 'fib_surf_pts'])

In [458]:
fig = px.scatter(width=800, height=1000)

fig.add_trace(go.Scatter3d(
x=tibia_l_transformed[tibia_l_lm_order['mrkrs_mscls'],0], 
y=tibia_l_transformed[tibia_l_lm_order['mrkrs_mscls'],1], 
z=tibia_l_transformed[tibia_l_lm_order['mrkrs_mscls'],2],
marker=dict(size=4, opacity=0.8), mode = 'markers + text',
name = 'markers and muscles',
#text=tibia_l_mrkrs_mscls_names,
textposition="top center"))


fig.add_trace(go.Scatter3d(
x=tibia_l_transformed[tibia_l_lm_order['wrp_rad'],0], 
y=tibia_l_transformed[tibia_l_lm_order['wrp_rad'],1], 
z=tibia_l_transformed[tibia_l_lm_order['wrp_rad'],2],
marker=dict(size=4, opacity=0.8), name = 'radius',
mode = 'markers'))

fig.add_trace(go.Scatter3d(x=tibia_l_transformed[[0,1],0], 
                           y=tibia_l_transformed[[0,1],1], 
                           z=tibia_l_transformed[[0,1],2],
                           name = "line between ankle points",
                           marker=dict(size=4, opacity=0.8)))

fig.add_trace(go.Scatter3d(x=[0,0], y=[0,0], z=[-35,35], name= 'left_right through 0,0,0', marker=dict(size=4, opacity=0.8)))

fig.add_trace(go.Scatter3d(
x=[tibia_l_transformed[tibia_l_lm_order['wrp_rad'][5],0], tibia_l_transformed[tibia_l_lm_order['wrp_transl'][5],0],tibia_l_transformed[tibia_l_lm_order['wrp_ax'][5],0]],
y=[tibia_l_transformed[tibia_l_lm_order['wrp_rad'][5],1],tibia_l_transformed[tibia_l_lm_order['wrp_transl'][5],1],tibia_l_transformed[tibia_l_lm_order['wrp_ax'][5],1]], 
z=[tibia_l_transformed[tibia_l_lm_order['wrp_rad'][5],2],tibia_l_transformed[tibia_l_lm_order['wrp_transl'][5],2],tibia_l_transformed[tibia_l_lm_order['wrp_ax'][5],2]],
marker=dict(size=4,opacity=0.8), name = 'line through rad, transl and ax'))

camera = dict(
    up=dict(x=0, y=1, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)

fig.update_scenes(aspectmode="data" )
fig.show()

In [459]:
# scale markers and muscles, create a dataframe for sorting and saving
body = 'tibia_l'
tibia_l_transformed_mrkr_mscl = {'body':[], 'name':[], 'location':[]}
for i, n in enumerate(tibia_l_mrkrs_mscls_names):
    tibia_l_transformed_mrkr_mscl['body'].append('tibia_l'),
    tibia_l_transformed_mrkr_mscl['name'].append(n),
    tibia_l_transformed_mrkr_mscl['location'].append(tibia_l_transformed[tibia_l_lm_order['mrkrs_mscls']][i]/1000)
tibia_l_transformed_mrkr_mscl_df = pd.DataFrame(tibia_l_transformed_mrkr_mscl)

## Collect all resulting data

In [460]:
## collect all markers and muscles
mrkrs_mscls_transformed_df = pd.concat([pelvis_transformed_mrkr_mscl_df, 
                                        femur_r_transformed_mrkr_mscl_df, 
                                        femur_l_transformed_mrkr_mscl_df, 
                                        patella_r_transformed_mrkr_mscl_df, 
                                        patella_l_transformed_mrkr_mscl_df, 
                                        tibia_r_transformed_mrkr_mscl_df, 
                                        tibia_l_transformed_mrkr_mscl_df])
mrkrs_mscls_transformed_df.set_index('name', inplace=True)
mrkrs_mscls_transformed_df.shape

(294, 2)

In [461]:
# create lists of markers and muscles that have been in the transformation
# in order to split the above dataframe
marker_names = list(mri_bone_markers.keys())
muscle_names = list(df_muscles['label'])
print('markers', len(marker_names), 'muscles', len(muscle_names))

markers 69 muscles 288


In [462]:
# now actually split the lists
mscls = []
mrkrs = []
missed = []
for name in list(mrkrs_mscls_transformed_df.index):
    if name in marker_names:
        mrkrs.append(name)
    elif name in muscle_names:
        mscls.append(name)
    else: missed.append(name)
mrkrs = mrkrs + missed

In [463]:
# create according dataframes and save them in csv files
mscls_transformed_df = mrkrs_mscls_transformed_df.loc[mscls]
mscls_transformed_df.to_csv('model_update/tps_warping_results/muscles_transformed.csv')

markers_transformed_df = mrkrs_mscls_transformed_df.loc[mrkrs]
markers_transformed_df.to_csv('model_update/tps_warping_results/markers_transformed.csv')

### Infer scaled parameters for muscle wrapping surfaces

In [120]:
#[pelvis_wrp, femur_r_wrp, femur_l_wrp, tibia_r_wrp, tibia_l_wrp]

#### pelvis

In [464]:
body = 'pelvis'
names_in_order = list(pelvis_wrp['name'])
transformed_landmarks = pelvis_transformed
landmark_order_dict = pelvis_lm_order
transformed_wrp_pelvis_df=return_transformed_wrapping_surfaces(transformed_landmarks, landmark_order_dict, names_in_order, body, mm_to_m = True)
# transformed_wrp_pelvis_df

#### femur_l

In [465]:
body = 'femur_l'
names_in_order = list(femur_l_wrp['name'])
transformed_landmarks = femur_l_transformed
landmark_order_dict = femur_l_lm_order
transformed_wrp_femur_l_df=return_transformed_wrapping_surfaces(transformed_landmarks, landmark_order_dict, names_in_order, body, mm_to_m = True)
# transformed_wrp_femur_l_df

#### femur_r

In [466]:
body = 'femur_r'
names_in_order = list(femur_r_wrp['name'])
transformed_landmarks = femur_r_transformed
landmark_order_dict = femur_r_lm_order
transformed_wrp_femur_r_df=return_transformed_wrapping_surfaces(transformed_landmarks, landmark_order_dict, names_in_order, body, mm_to_m = True)
# transformed_wrp_femur_r_df

#### tibia_r

In [467]:
body = 'tibia_r'
names_in_order = list(tibia_r_wrp['name'])
transformed_landmarks = tibia_r_transformed
landmark_order_dict = tibia_r_lm_order
transformed_wrp_tibia_r_df=return_transformed_wrapping_surfaces(transformed_landmarks, landmark_order_dict, names_in_order, body, mm_to_m = True)
#transformed_wrp_tibia_r_df

#### tibia_l

In [468]:
body = 'tibia_l'
names_in_order = list(tibia_l_wrp['name'])
transformed_landmarks = tibia_l_transformed
landmark_order_dict = tibia_l_lm_order
transformed_wrp_tibia_l_df=return_transformed_wrapping_surfaces(transformed_landmarks, landmark_order_dict, names_in_order, body, mm_to_m = True)
# transformed_wrp_tibia_l_df

In [469]:
wrp_transformed_df = pd.concat([transformed_wrp_pelvis_df, transformed_wrp_femur_r_df, transformed_wrp_femur_l_df,transformed_wrp_tibia_r_df,transformed_wrp_tibia_l_df])
wrp_transformed_df.to_csv('model_update/tps_warping_results/wrp_transformed.csv', index=False)

## Impute muscles to a scaled model
this does not guarantee the correct position of join centres. It only transfers muscles to the space of the 3DSlicer, where the muscles can be checked, corrected and then transformed together with bone markers to child body frame axes.